## Week 2 Day 3 - OpenAI Only Version

This is a duplicate of lab3 using only OpenAI models:

1. Different OpenAI models

2. Structured Outputs

3. Guardrails

In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import requests
import os
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

### Using different OpenAI models instead of external providers

In [5]:
# Using different OpenAI models instead of external APIs
sales_agent1 = Agent(name="GPT-4o Professional Sales Agent", instructions=instructions1, model="gpt-4o")
sales_agent2 = Agent(name="GPT-4o-mini Engaging Sales Agent", instructions=instructions2, model="gpt-4o-mini")
sales_agent3 = Agent(name="GPT-3.5-turbo Busy Sales Agent", instructions=instructions3, model="gpt-3.5-turbo")

In [6]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [7]:
@function_tool
def send_notification_with_title(subject: str, message: str) -> Dict[str, str]:
    """ Send out a push notification with the given subject and message to the sales team """
    pushover_user = os.environ.get('PUSHOVER_USER')
    pushover_token = os.environ.get('PUSHOVER_TOKEN')
    
    if not pushover_user or not pushover_token:
        raise RuntimeError("Missing PUSHOVER_USER or PUSHOVER_TOKEN in environment")

    def push(title, body):
        print(f"Push notification - Title: {title}, Message: {body}")
        payload = {
            "user": pushover_user, 
            "token": pushover_token, 
            "title": title,
            "message": body
        }
        response = requests.post("https://api.pushover.net/1/messages.json", data=payload)
        return response

    response = push(subject, message)
    response.raise_for_status()
    return {"status": "success"}

In [8]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [9]:
email_tools = [subject_tool, html_tool, send_notification_with_title]

In [10]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_notification_with_title tool to send a push notification with the subject and message body to the sales team."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it as a push notification")

In [11]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [12]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending as a push notification to the sales team.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR - OpenAI Only"):
    result = await Runner.run(sales_manager, message)

Push notification - Title: Transform Compliance into a Breeze – Discover How!, Message: Hi [CEO's Name],

I hope this email finds you knee-deep in innovation and not buried under a mountain of compliance paperwork! 🏔️

At ComplAI, we like to think of SOC2 compliance as that elusive sock that disappears in the laundry—annoying, but essential to find! Our AI-powered SaaS tool not only helps you track down those pesky compliance requirements, but it also makes preparing for audits feel like a walk in the park (complete with ice cream, of course! 🍦).

Imagine a world where audits are more breeze than burden! Our clients have experienced a delightful transformation in their compliance game, and I’d love to show you how we can sprinkle some of that magic into your business too.

How about a quick chat this week? I promise to keep it short and sweet (like a good dessert).

Looking forward to connecting!

Best regards,
Alice

P.S. You’ll be the envy of all your peers when compliance becomes yo

## Check out the trace:

https://platform.openai.com/traces

In [20]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

class EmailCheckOutput(BaseModel):
    is_email_in_message: bool
    email: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

email_guardrail_agent = Agent(
    name="Email check", 
    instructions="Check if the user is including an email address in what they want you to do. Look for patterns like user@domain.com or any text that appears to be an email address.",
    output_type=EmailCheckOutput,
    model="gpt-4o-mini"
)

In [21]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

@input_guardrail
async def guardrail_against_email(ctx, agent, message):
    result = await Runner.run(email_guardrail_agent, message, context=ctx.context)
    is_email_in_message = result.final_output.is_email_in_message
    return GuardrailFunctionOutput(output_info={"found_email": result.final_output},tripwire_triggered=is_email_in_message)

In [22]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name, guardrail_against_email]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR - OpenAI Only"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

## Check out the trace:

https://platform.openai.com/traces

In [23]:
message = (
    "Send out a cold sales email addressed to Dear CEO from Head of Business Development. Also include my email: abdullah@gmail.com."
)

with trace("Protected Automated SDR - OpenAI Only"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [24]:
message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR - OpenAI Only"):
    result = await Runner.run(careful_sales_manager, message)

Push notification - Title: Streamline Your SOC2 Compliance Effortlessly!, Message: A new cold sales email aimed at the CEO has been drafted and is ready for review and sending. Subject: 'Streamline Your SOC2 Compliance Effortlessly!'. The email body has been formatted in HTML for optimal presentation.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different OpenAI models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>